**CFL**

This is a complex algorithm for subgraph matching. It proposed a new structure to reduce the search sapce.

Also, this algorithm follow the classic three-step algorithm in the field of subgraph matching. The basic idea for this algorithm is that the authors want to reduce the search space of **non-tree edges**(the ones not in the BFS tree of the graph) by checking them as early as possible.

First, in the candidate set generation part, the authors use **forward passing and backward refining** to construct CPI(candidate set) for each node. And also, they generate a special structure **N[qnode][gnodeparent][gnode]**, where gnodes are the candidates of qnode and gnodeparent are the parent of gnode in the matching algorithm.

To construct CPI and N, the authors constructed **BFS tree** for the graph. For each level of the tree, we check the nodes in this level to see which ones in data graph could match with them. The forward candidate generation part mainly means: **a data vertex v is in u.C
only if for each $u^{'}$ $\in$ u.N, there is a data vertex $v^{'}$ $\in$ $u^{'}$.C that is adjacent to v.**(what the cnt for). After this, they use a backward candidate pruning: **for each query vertex u based on its set of unvisited neighbors (i.e., u.UN), which were not exploited in the forward processing.** Also, they construct **N** in this forward part by find parents for candidate set for each level of nodes.(backward pruning is considered in forward part). 

In the backward refining part, they **exploit the candidate sets of lower-level neighbors of u to prune unpromising candidates from u.C** like in the forward part. And also, they update **N** accordingly.

Then they try to generate a optimal matching order based on BFS tree we genereated before. They first classify the nodes into three structures: **core-structure, forest-structure, and leaf-structure** to reduce the search space by checking non-tree edges as early as possible(in the core part). They generate sub-orders for three structures separately using N they generated before. 

And finally, they use these three parts to enumerate to search the **corresponding map** and combine them(this way reduces the search space by removing some false path backward). For simplicity, I first combine the three sub-orders, and perform enumeration. The authors perfer to first iterate then combine(Easy to change with each other). 

The original enumeration part doesn't work correctly, so I change it like the one in the survey paper.

In [ ]:
!git clone https://github.com/RapidsAtHKUST/SubgraphMatching.git

fatal: destination path 'SubgraphMatching' already exists and is not an empty directory.


In [ ]:
from collections import defaultdict

In [ ]:
class graph():
  def __init__(self, graphid, node2label, node2degree, edges):
    self.graphid = graphid
    self.node2label = node2label
    self.node2degree = node2degree
    self.edges = edges
    self.candidateset = defaultdict(set)
    self.label2node = defaultdict(set)
    for node in self.node2label:
      self.label2node[self.node2label[node]].add(node)
    self.phi = []
    self.phiparent = {}

  def reset(self):
    self.candidateset = defaultdict(set)
    self.phi = []
    self.phiparent = {}

In [ ]:
def get_graph(filepath, filename):
  global qcount
  global gcount

  node2label = {}
  node2degree = {}
  edges = defaultdict(set)
  f = open(filepath, "r", encoding="utf-8")

  _, nodenum, edgenum = f.readline().strip().split()
  for i in range(int(nodenum)):
    _, nodeid, nodelabel, nodedegree = f.readline().strip().split()
    node2label[int(nodeid)] = int(nodelabel)
    node2degree[int(nodeid)] = int(nodedegree)  
  for i in range(int(edgenum)):
    _, node1, node2 = f.readline().strip().split()
    edges[int(node1)].add(int(node2))
    edges[int(node2)].add(int(node1))

  f.close()
  g = graph(filename, node2label, node2degree, edges)

  return g

In [ ]:
qcount = 0
gcount = 0

import os
qs = []
qdir = "SubgraphMatching/test/query_graph"
for f in os.listdir(qdir):
  filepath = os.path.join(qdir, f)
  qs.append(get_graph(filepath, f))

gs = []
gdir = "SubgraphMatching/test/data_graph"
for f in os.listdir(gdir):
  filepath = os.path.join(gdir, f)
  gs.append(get_graph(filepath, f))

print(len(qs))
print(len(gs))

f = open("SubgraphMatching/test/expected_output.res", "r", encoding="utf-8")
lines = f.readlines()
f.close()

expects = {}
for line in lines:
  name, times = line.strip().split(":")
  expects[name + ".graph"] = int(times)
print(len(expects))

200
1
200


In [ ]:
def CFL_CSG(q, g):
  vc = set()
  vt = set()
  vl = set()

  degrees = q.node2degree.copy()
  for node in degrees:
    vc.add(node)
    vl.add(node)
    vt.add(node)

  flag = True
  while(flag):
    flag = False
    for node in degrees:
      if degrees[node] == 1 and node in vc:
        vc.remove(node)
        flag = True
        for neighbor in q.edges[node]:
          if degrees[neighbor] > 0:
            degrees[neighbor] -= 1

  
  for node in q.node2degree:
    if q.node2degree[node] != 1 and node in vl:
      vl.remove(node)
  vl -= vc

  vt -= vc
  vt -= vl



  for qnode in vc:
    label = q.node2label[qnode]
    for gnode in g.label2node[label]:
      if q.node2degree[qnode] <= g.node2degree[gnode]:
        q.candidateset[qnode].add(gnode)
  
  roots = {}
  for qnode in vc:
    roots[qnode] = len(q.candidateset[qnode]) / q.node2degree[qnode]
  roots = sorted(roots.items(), key = lambda kv: kv[1])
  roots = roots[:3]
  rs = set()
  for root in roots:
    rs.add(root[0])
  
  qmaxneighbordegrees = {}
  for qnode in q.node2degree:
    maxdegree = 0
    for neighbor in q.edges[qnode]:
      maxdegree = max(maxdegree, q.node2degree[neighbor])
    qmaxneighbordegrees[qnode] = maxdegree
  
  gmaxneighbordegrees = {}
  for gnode in g.node2degree:
    maxdegree = 0
    for neighbor in g.edges[gnode]:
      maxdegree = max(maxdegree, g.node2degree[neighbor])
    gmaxneighbordegrees[gnode] = maxdegree
  
  for qnode in rs:
    for gnode in q.candidateset[qnode].copy():
      if gmaxneighbordegrees[gnode] < qmaxneighbordegrees[qnode]:
        q.candidateset[qnode].remove(gnode)

  
  roots = {}
  for r in rs:
    roots[r] = len(q.candidateset[r]) / q.node2degree[r]
  roots = sorted(roots.items(), key = lambda kv: kv[1])
  r = roots[0][0]

  q.reset()
  for gnode in g.label2node[q.node2label[r]]:
    if q.node2degree[r] <= g.node2degree[gnode]:
      if gmaxneighbordegrees[gnode] >= qmaxneighbordegrees[r]:
        q.candidateset[r].add(gnode)
  
  node2level = {}
  visited = set()
  queue = []
  visited.add(r)
  node2level[r] = 1
  maxlevel = 1
  queue.append(r)
  nodeparent = {}
  nodeparent[r] = -1
  nodechildren = defaultdict(set)
  while queue:
    top = queue[0]
    queue.pop(0)
    for neighbor in q.edges[top]:
      if neighbor not in visited:
        node2level[neighbor] = node2level[top] + 1
        nodeparent[neighbor] = top
        nodechildren[top].add(neighbor)
        maxlevel = max(maxlevel, node2level[neighbor])
        visited.add(neighbor)
        queue.append(neighbor)
  
  level2node = defaultdict(set)
  for node in node2level:
    level2node[node2level[node]].add(node)

  cnts = {}
  for gnode in g.node2label:
    cnts[gnode] = 0
  
  visited = set()
  visited.add(r)
  uns = defaultdict(set)
  n = defaultdict(lambda: defaultdict(set))
  for level in range(2, maxlevel + 1):
    qnodes = []
    for qnode in level2node[level]:
      qnodes.append(qnode)
      cnt = 0
      for qneighbor in q.edges[qnode]:
        if qneighbor not in visited and node2level[qneighbor] == level:
          uns[qnode].add(qneighbor)
        elif qneighbor in visited:
          for gnode in q.candidateset[qneighbor]:
            for gneighbor in g.edges[gnode]:
              if g.node2label[gneighbor] == q.node2label[qnode] and g.node2degree[gneighbor] >= q.node2degree[qnode]:
                if cnts[gneighbor] == cnt:
                  cnts[gneighbor] = cnt + 1
          cnt += 1

      for gnode in g.node2label:
        if cnts[gnode] == cnt:
          if gmaxneighbordegrees[gnode] >= qmaxneighbordegrees[qnode]:
            q.candidateset[qnode].add(gnode)
      visited.add(qnode)
      for gnode in cnts:
        if cnts[gnode] > 0:
          cnts[gnode] = 0

    qnodes.reverse()
    for qnode in qnodes:
      cnt = 0
      for qneighbor in uns[qnode]:
        for gnode in q.candidateset[qneighbor]:
          for gneighbor in g.edges[gnode]:
            if g.node2label[gneighbor] == q.node2label[qnode] and g.node2degree[gneighbor] >= q.node2degree[qnode]:
              if cnts[gneighbor] == cnt:
                cnts[gneighbor] = cnt + 1
        cnt += 1
      for gnode in q.candidateset[qnode].copy():
        if cnts[gnode] != cnt:
          q.candidateset[qnode].remove(gnode)
      for gnode in cnts:
        if cnts[gnode] > 0:
          cnts[gnode] = 0
  
    for qnode in level2node[level]:
      qnodeparent = nodeparent[qnode]
      for gnodeparent in q.candidateset[qnodeparent]:
        for gnode in g.edges[gnodeparent]:
          if g.node2label[gnode] == q.node2label[qnode] and gnode in q.candidateset[qnode]:
            n[qnode][gnodeparent].add(gnode)


  cnts = {}
  for gnode in g.node2label:
    cnts[gnode] = 0

  for level in range(maxlevel, 0, -1):
    for qnode in level2node[level]:
      cnt = 0
      for qneighbor in q.edges[qnode]:
        if node2level[qneighbor] > level:
          for gnode in q.candidateset[qneighbor]:
            for gneighbor in g.edges[gnode]:
              if g.node2label[gneighbor] == q.node2label[qnode] and g.node2degree[gneighbor] >= q.node2degree[qnode]:
                if cnts[gneighbor] == cnt:
                  cnts[gneighbor] = cnt + 1
          cnt += 1
      
      tmp = q.candidateset[qnode].copy()#not equal to the paper, the paper means deal with these two at the same time, so I have to use a tmp to store the inital candidateset

      for gnode in q.candidateset[qnode].copy():
        if cnts[gnode] != cnt:
          q.candidateset[qnode].remove(gnode)
    
      for gnode in cnts:
        if cnts[gnode] > 0:
          cnts[gnode] = 0


      for gnode in tmp:
        for qnodechild in nodechildren[qnode]:
          for gnodechild in n[qnodechild][gnode].copy():
            if gnodechild not in q.candidateset[qnodechild]:
              n[qnodechild][gnode].remove(gnodechild)


  return n, r, nodeparent, nodechildren, vc, vt, vl

In [ ]:
def CFL_MOG(q, n, r, nodeparent, nodechildren, vc, vt, vl):
  
  cpaths = []
  tpaths = []
  lpaths = []
  
  def dfs(cur, path):
    if len(nodechildren[cur] & vc) == 0:
      cpaths.append(path.copy())
      return
    
    for child in nodechildren[cur]:
      if child in vc:
        path.append(child)
        dfs(child, path)
        path.pop(-1)
  
  dfs(r, [r])
  
  def ccalculate(path):
    cs = {}
    for parent in n[path[-1]]:
      for child in n[path[-1]][parent]:
        cs[child] = 1

    for i in range(len(path) - 1, 0, -1):
      cur = path[i]
      for parent in n[cur]:
        cs[parent] = 0
        for child in n[cur][parent]:
          cs[parent] += cs[child]
    
    c = 0
    for node in cs:
      if node in q.candidateset[path[0]]:
        c += cs[node]
    
    return c

  def gen_order(paths, v):
    if not paths:
      return []
    cs = []
    for path in paths:
      cs.append(ccalculate(path) / (len(path) - 1))
    visited = set()
    minc = min(cs)
    minindex = 0
    addednodes = set()
    for i, c in enumerate(cs):
      if c == minc:
        visited.add(i)
        minindex = i
        break
    
    order = []
    for node in paths[i]:
      if node in v:
        order.append(node)
        addednodes.add(node)

    while len(visited) != len(paths):
      cs = {}
      for i, path in enumerate(paths):
        if i not in visited:
          j = 0
          while(j < len(path)):
            if path[j] not in addednodes:
              break
            j += 1
          cs[i] = ccalculate(path[j:]) / len(q.candidateset[path[j]])
      minc = cs[max(cs,key=cs.get)] + 1
      minindex = -1
      for i in cs:
        if i not in visited and cs[i] < minc:
          minc = c
          minindex = i

      visited.add(minindex)
      start = 0
      while(start < len(paths[minindex])):
        if paths[minindex][start] not in addednodes:
          break
        start += 1

      for i in range(start, len(paths[minindex])):
        if paths[minindex][i] in v:
          order.append(paths[minindex][i])
          addednodes.add(paths[minindex][i])
    return order
  
  def gen_path(v, vchild, paths):
    for node in v:
      for child in nodechildren[node]:
        path = []
        if child in vchild:
          cur = child
          path.append(cur)
          while len(nodechildren[cur] & vchild):
            cur = list(nodechildren[cur])[0]
            if cur in vchild:
              path.append(cur)
        if path:
          path.insert(0, node)
          paths.append(path)

  gen_path(vc, vt, tpaths)
  gen_path(vc, vl, lpaths)
  gen_path(vt, vl, lpaths)
  cpath = gen_order(cpaths, vc)
  tpath = gen_order(tpaths, vt)
  lpath = gen_order(lpaths, vl)
  q.phi = cpath + tpath + lpath
  
  for node in q.phi:
    q.phiparent[node] = nodeparent[node]

In [ ]:
def CFL_EP(q, g, m, i, totalresult): # not equal to the original code
  if i == len(q.phi) + 1:
    totalresult.append(m.copy())
    return 
  result = {}
  u = -1
  for node in q.phi:
    if node not in m:
      u = node
      break

  lc = set()

  if i == 1:
    lc = q.candidateset[u]
  elif q.phi.index(u) == 1:
    lc = n[u][m[nodeparent[u]]]
  else:
    for v in n[u][m[nodeparent[u]]]:
      flag = True
      for node in q.phi:
        if node == u:
          break
        if node == q.phiparent[u]:
          continue
        #if m[node] not in g.edges[v]:
        if v == m[node] or (m[node] not in g.edges[v] and (node in q.edges[u] or u in q.edges[node])):
          flag = False
          break
      if flag:
        lc.add(v)

  for node in lc:
    if node not in set(m.values()):
      m[u] = node
      CFL_EP(q, g, m, i + 1, totalresult)
      m.pop(u)

In [ ]:
queries = {}
for g in gs:
  for q in qs:
    q.reset()

    n, r, nodeparent, nodechildren, vc, vt, vl = CFL_CSG(q, g)
    CFL_MOG(q, n, r, nodeparent, nodechildren, vc, vt, vl)
    totalresult = []
    CFL_EP(q, g, {}, 1, totalresult)

    queries[q.graphid] = len(totalresult)

print(queries)

{'query_dense_16_41.graph': 8, 'query_dense_16_152.graph': 432, 'query_dense_16_44.graph': 12, 'query_dense_16_165.graph': 208, 'query_dense_16_77.graph': 4, 'query_dense_16_171.graph': 14, 'query_dense_16_62.graph': 8, 'query_dense_16_196.graph': 2, 'query_dense_16_60.graph': 10, 'query_dense_16_178.graph': 1, 'query_dense_16_100.graph': 16, 'query_dense_16_103.graph': 48, 'query_dense_16_136.graph': 1, 'query_dense_16_85.graph': 19, 'query_dense_16_40.graph': 12, 'query_dense_16_122.graph': 16, 'query_dense_16_172.graph': 16, 'query_dense_16_134.graph': 17, 'query_dense_16_67.graph': 1, 'query_dense_16_116.graph': 1, 'query_dense_16_10.graph': 32, 'query_dense_16_86.graph': 1, 'query_dense_16_22.graph': 9, 'query_dense_16_139.graph': 12, 'query_dense_16_89.graph': 12, 'query_dense_16_24.graph': 12, 'query_dense_16_71.graph': 9, 'query_dense_16_141.graph': 20, 'query_dense_16_151.graph': 138, 'query_dense_16_19.graph': 2, 'query_dense_16_36.graph': 3, 'query_dense_16_101.graph': 2, 'q

In [ ]:
flag = True
for name in expects:
  if expects[name] != queries[name]:
    print(name)
    flag = False
if flag:
  print("correct")

correct
